# Распознавание человека на видео

Методом face_recognition распознаем лицо актера Джими Керри на фрагменте видео из фильма "Шоу Тремана". Из кадров с актером соберем новый фильм и посчитаем его длительность.

# Загрузка данных


In [1]:
pip install face_recognition

     |████████████████████████████████| 100.2MB 50kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566184 sha256=07ab7dba1614197ceef871e26efc66114ab259e6bb259597609de477459337a9
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [19]:
import os
import pickle
import sys
import face_recognition
from cv2 import cv2
from google.colab.patches import cv2_imshow
import time

from IPython.display import HTML
from base64 import b64encode

from PIL import Image, ImageDraw

In [3]:
from google.colab import drive
drive.mount('/content/drive')
directory = '/content/drive/MyDrive/Colab Notebooks/Neural_university/Соревнование/Carrey/'

Mounted at /content/drive


# Тренировка на фото

Предварительно из видео вырезаны примеры изображений Керри и помещены в папку "img_carrey".
Задача - создать файл с обобщенными данными на изображение  Керри carrey_encodings.pickle


In [108]:
# Скрипт получает изображение, делает скриншот и отправляет на сервер. Сравниват с базой и если сопадает то делает действие
# Имя человека, которого определяем
def train_model_by_img(name_acter, folder_carrey):

    if not os.path.exists(folder_carrey):
        print("[ERROR] there is no directory 'img_carrey'")
        sys.exit()

    known_encodings = []
    num_img_carrey = 0
    # Здесь несколько снимков одного человека img_carrey
    images = os.listdir(folder_carrey)
    print('Кадров для обучения:', len(images))   # Выведем имена файлов
    
    # Пробежимся по каждому изображени, загружакем в face_recognitions, снимаем кодировку и сравниваем
    # enumerate позволит кроме изображения получить его индекс - для выврода в терминал чтобы видетиь програсс
    for (i, image) in enumerate(images):     
        # print(f"[+] распознаем {i} изображение, файл - {image} из {len(images)}")
        face_img = face_recognition.load_image_file(folder_carrey + image)    #Загрузка изображдений        
        try:
          face_enc = face_recognition.face_encodings(face_img)[0]      #Кодировка каждого лица

          # Каждая полученная кодировка должна сопосталяться с полученной ранее
          # Если True - то добавляем в список
          # перовначально он пустой и добавляем туда первую кодировку
          if len(known_encodings) == 0:
              known_encodings.append(face_enc)
          
          # Если там уже естm первая кодировка, то проходим по этой кодироdке
          else:
            # face_enc - новое изображение сравниваем соо всеми в списке. Может быт долго если длинный список
              for item in range(0, len(known_encodings)):        
                  result = face_recognition.compare_faces([face_enc], known_encodings[item])
                  #print(result)      # Выводим узнал или нет Керри

                  if result[0]:
                      known_encodings.append(face_enc)     # Если True - добавляем в список и выходим из списка
                      #print("Тот же человек")
                      num_img_carrey += 1
                      break
                  else:
                      #print("Другой человек!")
                      break
        except:
         print('не узнал')
    #print(known_encodings)
    #print(f"Length {len(known_encodings)}")
    print('Узнал фоток', num_img_carrey)
    data = {
        "name": name_acter,
        "encodings": known_encodings
    }

    with open(directory + f"{name_acter}_encodings.pickle", "wb") as file:    # wb - запись в двоичном формате
        file.write(pickle.dumps(data))

    return f"[INFO] File {name_acter}_encodings.pickle successfully created"



# Распознавание человека

С помошью файла carrey_encodings.pickle сравниваемобобщенный образ актера с кадрами фильма и сохраняем кадры, где ст актер в папку.

In [109]:
# Определим Керри на видео и сохраним эти кадры

def detect_person_in_video(name_video, foldet_scrinshots):
    data = pickle.loads(open(directory + 'carrey_encodings.pickle', "rb").read())    # Читаем в двоичном формате
    video = cv2.VideoCapture(name_video)          # Можноб сюда данные из камеры
    num = 0

    while True:
        ret, image = video.read()              # Читаем и декодируем кадр
        
        # cnn - для работы на gpu
        locations = face_recognition.face_locations(image, model="cnn")           # Получим массив координат лиц на кадре
        encodings = face_recognition.face_encodings(image, locations)             # получим кодировки лиц из кадров
 
        # Сравнение лиц из датасета с лицами из каждого кадра. Если совпадает - выделяем лицо на видео
        # Циклом проходим по кодировкам и локациям
        for face_encoding, face_location in zip(encodings, locations):         
            # Для сравнения кодировки лиц. Первое - из датасета, вторая из кадра
            result = face_recognition.compare_faces(data["encodings"], face_encoding)    
            match = None                          # Совпадение

            if True in result:                  # Если совпадает
                match = data["name"]
                #print(f"Он найден! {match}")
                cv2.imwrite(foldet_scrinshots + f"/carrey{num}.jpg", image)
            #else:
                #print("Увы!")
        num += 1


# Создание видео и кадров
Метолом библиотеки cv2.VideoWriter создаем выходной видеофайл и записываем туда кадры с актером из папки.
Подсчитываем дилительность видео

In [110]:
# Создадим видео и кадров с Керри
def save_video(name_video, name_out_video, foldet_scrinshots):
  num_images = os.listdir(foldet_scrinshots)   # загрузим все файлы скриншотов

  # Установим формат выходного видео mp4
  fourcc = cv2.VideoWriter_fourcc(*'MJPG')
  video = cv2.VideoCapture(name_video)    # Определим частоту кадров fps в исходном видео чтобы в новом сделать такое же

  fps = int(video.get(cv2.CAP_PROP_FPS))       # Определим число кадро в сек. в исходном видео
  width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))    # Определим ширину и высоту кадра
  height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
  frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))   # Определим число кадров
  print('fps = ' + str(fps))
  print('frame_count исходного видео', frame_count)

  out_video = cv2.VideoWriter(name_out_video, fourcc, fps, (width, height)) # создаем видео
  for i in num_images:
    out_video.write(cv2.imread(foldet_scrinshots + i))  #добавляем картинки из директории


  # Для получившегося видео найдем его продолжительность. Это будет экранное время актера
  # fps такой же как и у исходного видео
  
  # определим число кадров. Но т.к. файл с видео записывается в колаб примерно через 1-2 мин., 
  # то вместо метода cv2 воспользуемся методом подсчета числа кадров в директории
  frame_count = len(num_images)
  print('frame_count получившегося видео', frame_count)
  duration = round(frame_count/fps, 3)

  print('Продолжительность созданного видео = ' + str(duration) + ' сек')
  minutes = int(duration/60)
  seconds = duration%60
  print('Продолжительность созданного видео = ' + str(minutes) + ' мин, ' + str(seconds) + ' сек')

  # конвертируем в mp4, как просят по заданию
  os.popen("ffmpeg -i '{input}' -ac 2 -b:v 2000k -c:a aac -c:v libx264 -b:a 160k -vprofile high -bf 0 -strict experimental -f mp4 '{output}.mp4'".format(input = name_out_video, output = name_out_video))

  out_video.release() #генерируем
  cv2.destroyAllWindows() #завершаем

  return



# Главная программа

In [112]:
def main():
    
    name_video = directory + "Shou_Trumana_Kinosimka.RU (online-video-cutter.com).mp4"    # Имя файла исходного видео
    name_out_video = directory + 'out_video_carry_full-4.avi'    # Имя файлы получившегося видео с актером
    foldet_scrinshots = directory + 'frame_carrey/'  # Папка для хранения всех найденных кадров с актером frame_carrey3 - для тренировочного видео
    folder_carrey = directory + 'img_carrey/'     # Папка для хранения образцов лица актера
    name_acter = 'carrey'

    print('Тренируем скрипт..')
    train_model_by_img(name_acter, folder_carrey)    # Если файл carrey_encodings.pickle уже есть, то тренировать заново не надо
    
    try:
      print('Распознаем Керри..')
      detect_person_in_video(name_video, foldet_scrinshots)
    except:
      print('Распознали')
    
    print('Записываем видео...')
    save_video(name_video, name_out_video, foldet_scrinshots)
    print('Приятного просмотра!')


if __name__ == '__main__':
    main()


Тренируем скрипт..
Кадров для обучения: 28
не узнал
Узнал фоток 22
Распознаем Керри..
Распознали
Записываем видео...
fps = 23
frame_count исходного видео 29629
frame_count получившегося видео 9678
Продолжительность видео, сек 420.783
Продолжительность видео, мин : сек = 7:0.7830000000000155
Приятного просмотра!
